# Imports

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.inspection import permutation_importance
import ta

from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.metrics import f1_score
from imblearn.under_sampling import RandomUnderSampler
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(suppress=True)
from pprint import pprint

import gc
import optuna
from sklearn.feature_selection import RFE


/tmp/ipykernel_26259/384002432.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('Data/Training/pair_features_1_pairs1_300_120.csv')
df.head()

,Ticker_P1,Date,High_P1,Low_P1,Volume_P1,Close_P1,Ticker_P2,High_P2,Low_P2,Volume_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_MA,abs_spread_std_MA,pnls,actual_abs_spread,actual_abs_spread_std
0,UNH,2001-02-09 00:00:00,14.962,14.562,9462800.000,11.819,COP,21.288,20.994,8208926.000,10.112,1.706,1.574,0.787,1.430,0.749,0.000,1.615,0.413
1,UNH,2001-02-12 00:00:00,15.123,14.507,10798000.000,11.934,COP,21.600,21.345,10666435.000,10.219,1.714,1.570,0.783,1.469,0.742,0.000,1.621,0.415
2,UNH,2001-02-13 00:00:00,15.188,14.632,6012400.000,11.827,COP,21.798,21.345,7761867.000,10.368,1.459,1.565,0.779,1.516,0.709,0.000,1.635,0.414
3,UNH,2001-02-14 00:00:00,15.017,14.222,7660000.000,11.506,COP,21.783,21.295,12059819.000,10.259,1.247,1.559,0.774,1.549,0.678,0.000,1.655,0.405
4,UNH,2001-02-15 00:00:00,14.700,14.060,11206400.000,11.354,COP,21.650,21.250,10194190.000,10.176,1.179,1.552,0.768,1.580,0.642,0.000,1.658,0.401


In [3]:
df['pnls'] = df['pnls']*100.0
# df = df[df.pnls != 0]

In [4]:
total_days = df.Date.nunique()
total_days

5738

In [5]:
# df.dropna()

In [6]:
df.pnls.describe()

count   5738.000
mean       0.203
std        2.630
min      -13.353
25%        0.000
50%        0.000
75%        0.521
max       25.707
Name: pnls, dtype: float64

## Technical Indicators

In [7]:
df

,Ticker_P1,Date,High_P1,Low_P1,Volume_P1,Close_P1,Ticker_P2,High_P2,Low_P2,Volume_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_MA,abs_spread_std_MA,pnls,actual_abs_spread,actual_abs_spread_std
0,UNH,2001-02-09 00:00:00,14.962,14.562,9462800.000,11.819,COP,21.288,20.994,8208926.000,10.112,1.706,1.574,0.787,1.430,0.749,0.000,1.615,0.413
1,UNH,2001-02-12 00:00:00,15.123,14.507,10798000.000,11.934,COP,21.600,21.345,10666435.000,10.219,1.714,1.570,0.783,1.469,0.742,0.000,1.621,0.415
2,UNH,2001-02-13 00:00:00,15.188,14.632,6012400.000,11.827,COP,21.798,21.345,7761867.000,10.368,1.459,1.565,0.779,1.516,0.709,0.000,1.635,0.414
3,UNH,2001-02-14 00:00:00,15.017,14.222,7660000.000,11.506,COP,21.783,21.295,12059819.000,10.259,1.247,1.559,0.774,1.549,0.678,0.000,1.655,0.405
4,UNH,2001-02-15 00:00:00,14.700,14.060,11206400.000,11.354,COP,21.650,21.250,10194190.000,10.176,1.179,1.552,0.768,1.580,0.642,0.000,1.658,0.401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5733,UNH,2023-11-24 00:00:00,547.590,542.810,1135000.000,545.240,COP,116.570,114.860,2159200.000,114.672,430.568,388.372,19.988,420.102,6.477,1.261,425.091,13.099
5734,UNH,2023-11-27 00:00:00,550.890,541.530,2675800.000,541.742,COP,115.130,113.360,5289500.000,113.987,427.755,388.444,20.092,421.164,5.840,1.261,423.796,14.019
5735,UNH,2023-11-28 00:00:00,545.820,540.070,2859300.000,538.692,COP,115.980,114.480,3469100.000,114.166,424.527,388.497,20.166,421.817,5.414,1.261,422.826,14.594
5736,UNH,2023-11-29 00:00:00,541.860,528.650,4550000.000,533.161,COP,115.560,113.320,6065300.000,113.163,419.998,388.535,20.214,421.998,5.281,1.261,422.288,14.884


### Beta

In [8]:
def compute_beta(rs,df,ticker,market):
    rows = df.loc[rs.index]
    sec_returns = np.log( df[[ticker,market]] / df[[ticker,market]].shift(1) ) 

    cov = sec_returns.cov() * 250
    cov_with_market = cov.iloc[0,1]
    market_var = sec_returns[market].var() * 250
    beta = cov_with_market / market_var
    return beta

In [9]:
all_tickers = df['Ticker_P1'].unique().tolist()+df['Ticker_P2'].unique().tolist()
len(all_tickers)

2

### Single Ticker Features

In [10]:
df.columns

Index(['Ticker_P1', 'Date', 'High_P1', 'Low_P1', 'Volume_P1', 'Close_P1',
       'Ticker_P2', 'High_P2', 'Low_P2', 'Volume_P2', 'Close_P2', 'abs_spread',
       'abs_spread_mean', 'abs_spread_std', 'abs_spread_mean_MA',
       'abs_spread_std_MA', 'pnls', 'actual_abs_spread',
       'actual_abs_spread_std'],
      dtype='object')

In [11]:
df.sort_values('Date')

,Ticker_P1,Date,High_P1,Low_P1,Volume_P1,Close_P1,Ticker_P2,High_P2,Low_P2,Volume_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_MA,abs_spread_std_MA,pnls,actual_abs_spread,actual_abs_spread_std
0,UNH,2001-02-09 00:00:00,14.962,14.562,9462800.000,11.819,COP,21.288,20.994,8208926.000,10.112,1.706,1.574,0.787,1.430,0.749,0.000,1.615,0.413
1,UNH,2001-02-12 00:00:00,15.123,14.507,10798000.000,11.934,COP,21.600,21.345,10666435.000,10.219,1.714,1.570,0.783,1.469,0.742,0.000,1.621,0.415
2,UNH,2001-02-13 00:00:00,15.188,14.632,6012400.000,11.827,COP,21.798,21.345,7761867.000,10.368,1.459,1.565,0.779,1.516,0.709,0.000,1.635,0.414
3,UNH,2001-02-14 00:00:00,15.017,14.222,7660000.000,11.506,COP,21.783,21.295,12059819.000,10.259,1.247,1.559,0.774,1.549,0.678,0.000,1.655,0.405
4,UNH,2001-02-15 00:00:00,14.700,14.060,11206400.000,11.354,COP,21.650,21.250,10194190.000,10.176,1.179,1.552,0.768,1.580,0.642,0.000,1.658,0.401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5733,UNH,2023-11-24 00:00:00,547.590,542.810,1135000.000,545.240,COP,116.570,114.860,2159200.000,114.672,430.568,388.372,19.988,420.102,6.477,1.261,425.091,13.099
5734,UNH,2023-11-27 00:00:00,550.890,541.530,2675800.000,541.742,COP,115.130,113.360,5289500.000,113.987,427.755,388.444,20.092,421.164,5.840,1.261,423.796,14.019
5735,UNH,2023-11-28 00:00:00,545.820,540.070,2859300.000,538.692,COP,115.980,114.480,3469100.000,114.166,424.527,388.497,20.166,421.817,5.414,1.261,422.826,14.594
5736,UNH,2023-11-29 00:00:00,541.860,528.650,4550000.000,533.161,COP,115.560,113.320,6065300.000,113.163,419.998,388.535,20.214,421.998,5.281,1.261,422.288,14.884


In [12]:
df.columns

Index(['Ticker_P1', 'Date', 'High_P1', 'Low_P1', 'Volume_P1', 'Close_P1',
       'Ticker_P2', 'High_P2', 'Low_P2', 'Volume_P2', 'Close_P2', 'abs_spread',
       'abs_spread_mean', 'abs_spread_std', 'abs_spread_mean_MA',
       'abs_spread_std_MA', 'pnls', 'actual_abs_spread',
       'actual_abs_spread_std'],
      dtype='object')

In [13]:
label = 'actual_abs_spread'
exclude = ['pnls', 'High_P1', 'High_P2', 'Low_P1', 'Low_P2', 'abs_spread', 'abs_spread_mean', 'abs_spread_std', 'actual_abs_spread_std']
X = df
y = df[label]

X = X.drop(['Date', 'Ticker_P1', 'Ticker_P2', 'Volume_P1', 'Volume_P2', label],axis=1)
X = X.drop(exclude, axis=1)

In [14]:
X = X.dropna()
X['abs_spread_log_mean_MA'] = np.log(X['abs_spread_mean_MA'])
X['abs_spread_log_std_mean_MA'] = np.log(X['abs_spread_std_MA'])
X = X.drop(['abs_spread_mean_MA', 'abs_spread_std_MA'],axis=1)
X

,Close_P1,Close_P2,abs_spread_log_mean_MA,abs_spread_log_std_mean_MA
0,11.819,10.112,0.358,-0.289
1,11.934,10.219,0.385,-0.298
2,11.827,10.368,0.416,-0.344
3,11.506,10.259,0.438,-0.389
4,11.354,10.176,0.458,-0.443
...,...,...,...,...
5733,545.240,114.672,6.040,1.868
5734,541.742,113.987,6.043,1.765
5735,538.692,114.166,6.045,1.689
5736,533.161,113.163,6.045,1.664


In [15]:
y = np.log(y)
y.describe()

count   5738.000
mean       3.190
std        1.837
min       -1.394
25%        2.002
50%        2.692
75%        5.093
max        6.081
Name: actual_abs_spread, dtype: float64

## Train test split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42,shuffle=False)

In [17]:
print(df.loc[X_train.index].Date.min(),df.loc[X_train.index].Date.max())

2001-02-09 00:00:00 2019-05-10 00:00:00


In [18]:
print(df.loc[X_val.index].Date.min(),df.loc[X_val.index].Date.max())

2019-05-13 00:00:00 2021-08-19 00:00:00


In [19]:
print(df.loc[X_test.index].Date.min(),df.loc[X_test.index].Date.max())

2021-08-20 00:00:00 2023-11-30 00:00:00


## LSTM Model

In [20]:
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout
#Creating the LSTM model
class lstm(Model):
    def __init__(self, in_shape):
        super(lstm, self).__init__()
        self.shape = in_shape
        self.lstm = LSTM(512, input_shape=self.shape, dropout=0.2, return_sequences = True)
        self.dropout = Dropout(0.2)
        self.dense = Dense(1)

    def call(self, x):
        x = self.lstm(x)
#         x = self.dropout(x)
        x = self.dense(x)
        return x

2024-03-20 06:47:13.961872: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 06:47:14.703862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [21]:
# Reshape X for model training
trainX = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1])).astype(np.float32)
valX = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1])).astype(np.float32)
testX = np.reshape(X_test, ( X_test.shape[0], 1, X_test.shape[1])).astype(np.float32)

In [22]:
trainX.shape

(4590, 1, 4)

In [ ]:
#initializing the lstm model
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=5,\
                                            min_delta=0.001,restore_best_weights=True,verbose=1)

lstm_model = lstm((trainX.shape[0], 1, trainX.shape[1]))
lstm_model.compile(loss='mae', 
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002), metrics=['mae'])
#fitting the model
history = lstm_model.fit(
                    trainX, 
                    y_train,
                    validation_data=(valX, y_val), 
                    epochs=20, 
                    batch_size=32,
                    verbose=1, 
                    callbacks=callback,
                    shuffle=False)       

Epoch 1/20


2024-03-20 06:47:15.512468: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


144/144 [==============================] - 4s 14ms/step - loss: 0.8335 - mae: 0.8335 - val_loss: 0.3692 - val_mae: 0.3692
Epoch 2/20
144/144 [==============================] - 2s 12ms/step - loss: 0.5997 - mae: 0.5997 - val_loss: 0.4572 - val_mae: 0.4572
Epoch 3/20
144/144 [==============================] - 2s 12ms/step - loss: 0.5668 - mae: 0.5668 - val_loss: 0.3341 - val_mae: 0.3341
Epoch 4/20
144/144 [==============================] - 2s 12ms/step - loss: 0.5300 - mae: 0.5300 - val_loss: 0.3188 - val_mae: 0.3188
Epoch 5/20
144/144 [==============================] - 2s 12ms/step - loss: 0.5050 - mae: 0.5050 - val_loss: 0.2374 - val_mae: 0.2374
Epoch 6/20
144/144 [==============================] - 2s 12ms/step - loss: 0.4757 - mae: 0.4757 - val_loss: 0.1924 - val_mae: 0.1924
Epoch 7/20
144/144 [==============================] - 2s 12ms/step - loss: 0.4602 - mae: 0.4602 - val_loss: 0.1681 - val_mae: 0.1681
Epoch 8/20
144/144 [==============================] - 2s 12ms/step - loss: 0.459

In [ ]:
lstm_model.summary()

In [ ]:
# Plot line graph to show amount loss according the the epoch
plt.figure(figsize=(8,6))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Loss Plot -- LSTM', fontsize=14, color="black", fontweight="bold")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
inference_result = lstm_model.predict(testX).flatten()

In [ ]:
y_test.plot.hist(bins=12, alpha=0.5)
y_test.describe()

In [ ]:
pd.DataFrame(inference_result).plot.hist(bins=12, alpha=0.5)
pd.Series(inference_result).describe()

In [ ]:
lstm_mae = (inference_result[:len(y_test)] - y_test).abs().mean()
lstm_mae

In [ ]:
inference_result

In [ ]:
mae = lstm_mae / y_test.abs().mean()
mae

In [ ]:
# Generate the PNL from predicted spread
from pair_trading_foundations.data_generation import ExecutePairTrading

BacktestData = df.loc[X_test.index]

samples = BacktestData
samples = samples.drop(['Date', 'Ticker_P1', 'Ticker_P2', 'Volume_P1', 'Volume_P2', label],axis=1)
samples = samples.drop(exclude,axis=1)
samples['abs_spread_log_mean_MA'] = np.log(samples.abs_spread_mean_MA)
samples['abs_spread_log_std_mean_MA'] = np.log(samples.abs_spread_std_MA)
samples = samples.drop(['abs_spread_mean_MA', 'abs_spread_std_MA'],axis=1)

model_input = np.reshape(samples, (samples.shape[0], 1, samples.shape[1])).astype(np.float32)
inference_result = lstm_model.predict(model_input).flatten()
BacktestData['pred_abs_spread'] = np.exp(inference_result)
BacktestData['diff_abs_spread'] = BacktestData['pred_abs_spread'] - BacktestData['abs_spread_mean_MA']
BacktestData['z_score'] = (np.log(BacktestData['pred_abs_spread']) / BacktestData['abs_spread_std_MA'])
BacktestData = BacktestData.reset_index()
BacktestData

pnls = []
entry_signal = 2
exit_signal = 0.5
test_len = 20
for idx in range(BacktestData.shape[0]):
    if (idx > BacktestData.shape[0]-test_len-1):
        pnls.append(np.nan)
    else:
        current_row = BacktestData.loc[idx]
        result = ExecutePairTrading(current_row.pred_abs_spread,   # Use predicted spread
                                    current_row.abs_spread_std_MA, # Use current std deviation
                                    entry_signal=entry_signal,
                                    exit_signal=exit_signal
                                ).execute(
                                    # Forward window
                                    vec1=BacktestData.loc[(idx+1):(idx+test_len)]['Close_P1'].values,
                                    vec2=BacktestData.loc[(idx+1):(idx+test_len)]['Close_P2'].values,
                                    dates=BacktestData.loc[(idx+1):(idx+test_len)]['Date'].values,
                                    base_fund=100,
                                )

        pnls.append(result.final_pl_pct)

BacktestData['pnls'] = pnls
BacktestData = BacktestData.dropna()
BacktestData

strategy = BacktestData.loc[(abs(BacktestData['z_score']) > 0.95)]
strategy

In [ ]:
pd.DataFrame(strategy.pnls).plot.hist(bins=12, alpha=0.5)
pd.Series(strategy.pnls).describe()

In [ ]:
# Normality test

from scipy.stats import shapiro
# normality test
stat, p = shapiro(inference_result)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

In [ ]:
# D’Agostino’s K^2 Test
from scipy.stats import normaltest
# normality test
stat, p = normaltest(inference_result)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

In [ ]:
# Anderson-Darling Test
from scipy.stats import anderson
result = anderson(inference_result)
print('Statistic: %.3f' % result.statistic)
p = 0
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
if result.statistic < result.critical_values[i]:
    print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl, cv))
else:
    print('%.3f: %.3f, data does not look normal (reject H0)' % (sl, cv))